## CAUSE OF FIRE CLASSIFIER

Classifying causes of fires based on size, location and day of the year. 

Main focus of classification is to maximize recall, as we want to find as much of suspicious fires, where human intervention or negligence was the cause of fire, and it should be investigated.

Choosing relevant features, feature engineering and searching for the best clasiifier

In [28]:
import pandas as pd
import numpy as np
import sqlite3
import os
from datetime import date, timedelta
import datetime
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as ss
# preprocessing, model selections, metrics
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score,RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.metrics import accuracy_score,f1_score, multilabel_confusion_matrix

# classificators
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.linear_model import LinearRegression

from scripts import find_best_model

In [2]:
# database connection
con = sqlite3.connect("FPA_FOD_20170508.sqlite")

In [3]:
# loading data - I limited data to one year to simplify computations and to limit memory and processor usage, to match my computers' limits
data = pd.read_sql_query("SELECT * FROM Fires WHERE FIRE_YEAR = 2014",con)
data.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1721941,300000001,FS-1524899,FED,FS-FIRESTAT,FS,USIDNCF,Nez Perce - Clearwater National Forests,0117,Nezperce National Forest,...,A,45.340833,-116.466667,5.0,USFS,ID,Idaho,049,Idaho,b'\x00\x01\xad\x10\x00\x00\x1cr\xe1\xdd\xdd\x1...
1,1721942,300000002,FS-1524894,FED,FS-FIRESTAT,FS,USIDNCF,Nez Perce - Clearwater National Forests,0117,Nezperce National Forest,...,A,45.505278,-116.425556,5.0,USFS,ID,Idaho,049,Idaho,"b'\x00\x01\xad\x10\x00\x00,5cM<\x1b]\xc0p?:\xf..."
2,1721943,300000003,FS-1524891,FED,FS-FIRESTAT,FS,USIDNCF,Nez Perce - Clearwater National Forests,0117,Nezperce National Forest,...,A,45.908056,-115.767778,5.0,USFS,ID,Idaho,049,Idaho,b'\x00\x01\xad\x10\x00\x00\x80\xeciE#\xf1\\\xc...
3,1721944,300000004,FS-1524900,FED,FS-FIRESTAT,FS,USIDNCF,Nez Perce - Clearwater National Forests,0117,Nezperce National Forest,...,A,45.840000,-115.966389,5.0,USFS,ID,Idaho,049,Idaho,b'\x00\x01\xad\x10\x00\x00 q\xc9P\xd9\xfd\\\xc...
4,1721945,300000005,FS-1522737,FED,FS-FIRESTAT,FS,USIDNCF,Nez Perce - Clearwater National Forests,0117,Nezperce National Forest,...,A,45.598333,-115.449167,5.0,USFS,ID,Idaho,049,Idaho,b'\x00\x01\xad\x10\x00\x00\x98\x86\x8f%\xbf\xd...


In [55]:
# viewing unique causes of fires (target)
causes = {}
for i in range(len(data.STAT_CAUSE_CODE.unique())):
    causes[i+1] = data[data.STAT_CAUSE_CODE==i+1]["STAT_CAUSE_DESCR"].iloc[0]

In [56]:
causes

{1: 'Lightning',
 2: 'Equipment Use',
 3: 'Smoking',
 4: 'Campfire',
 5: 'Debris Burning',
 6: 'Railroad',
 7: 'Arson',
 8: 'Children',
 9: 'Miscellaneous',
 10: 'Fireworks',
 11: 'Powerline',
 12: 'Structure',
 13: 'Missing/Undefined'}

In [4]:
# computing the length of fire
data["time_burning"] = [(data.CONT_DATE[i]-data.DISCOVERY_DATE[i])+1 for i in range(len(data.CONT_DATE))]

In [7]:
# filling missing values
data["time_burning"] = data.time_burning.fillna(data["time_burning"].mean())

In [8]:
# selecting columns based on descriptions

data_classification = data[["DISCOVERY_DOY","STAT_CAUSE_CODE","FIRE_SIZE","STATE","COUNTY","LONGITUDE","LATITUDE","OWNER_CODE","time_burning"]]

In [9]:
# checking NaN values
data_classification.isna().sum()

DISCOVERY_DOY         0
STAT_CAUSE_CODE       0
FIRE_SIZE             0
STATE                 0
COUNTY             6361
LONGITUDE             0
LATITUDE              0
OWNER_CODE            0
time_burning          0
dtype: int64

In [10]:
# translating state names to numbers
for i in range(len(data_classification.STATE.unique())):
        data_classification.STATE = data_classification.STATE.replace(data_classification.STATE.unique()[i],str(i))

C:\Users\szklarnia\anaconda3\lib\site-packages\pandas\core\generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1/3 of county data is missing. We cannot assign most common name to NaN's or fill them in some other way. Column will be dropped, LAT and LONG combined carry simmilar, even more detailed  information

In [11]:
# specyfing learning set and target
X = data_classification[["DISCOVERY_DOY","FIRE_SIZE","STATE","LONGITUDE","LATITUDE","OWNER_CODE"]]
y = data_classification["STAT_CAUSE_CODE"]

In [12]:
# performing test train split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [13]:
X_train

,DISCOVERY_DOY,FIRE_SIZE,STATE,LONGITUDE,LATITUDE,OWNER_CODE
33040,93,0.3,22,-79.419060,34.591179,8.0
17383,175,2.0,11,-104.187960,31.039060,14.0
45683,21,12.0,17,-87.557546,33.948415,14.0
62005,50,10.0,11,-97.153045,33.254397,14.0
772,4,0.1,4,-118.750833,34.644167,5.0
...,...,...,...,...,...,...
32780,92,0.2,21,-78.220617,35.993850,14.0
13890,201,0.1,3,-111.936900,39.196100,1.0
35091,124,1.0,31,-91.472500,37.634444,14.0
26882,20,13.0,18,-88.762417,36.153700,8.0


In [14]:
# trying simple logistic regression to see if data is possible to model

LR = Pipeline([("Scaler",StandardScaler()),("Logistic_Regression",LogisticRegression(solver="liblinear"))])

LR.fit(X_train,y_train)
LR.score(X_test,y_test)

0.4254298575750867

In [15]:
# we'll try 3 classifiers at first - Logistic Regression, Decision Tree and Random Forest
pipelines = [make_pipeline(StandardScaler(),LogisticRegression(solver="liblinear")), # Logistic Regression
            make_pipeline(StandardScaler(),DecisionTreeClassifier()),                # Decision Tree
            make_pipeline(StandardScaler(),RandomForestClassifier())]                     # Random Forest

# Defining parameters to check for each defined model. I'm using only gini criterion for random forest and decision tree,
# as it's computation is faster and i have a limited amount of computational power
param_grids = [{"logisticregression__penalty":["l1","l2"],"logisticregression__C":[0.001,0.01,0.1,1,10,100,1000]},
              {"decisiontreeclassifier__criterion":["gini"],"decisiontreeclassifier__min_samples_split":[2,4,6,8,10]},
              {"randomforestclassifier__criterion":["gini"],
              "randomforestclassifier__n_estimators":[100,200,300], "randomforestclassifier__min_samples_split":[2,4,6,8,10]}]

In [16]:
# finding best model using GridSearchCV

find_best_model(X_train,X_test,y_train,y_test,pipelines,param_grids)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   14.8s
[Parallel(n_jobs=3)]: Done  70 out of  70 | elapsed:   47.2s finished
C:\Users\szklarnia\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=3)]: Done  25 out of  25 | elapsed:    3.2s finished


Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.5min
[Parallel(n_jobs=3)]: Done  75 out of  75 | elapsed:  6.5min finished


Best model: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())]) with {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__min_samples_split': 6, 'randomforestclassifier__n_estimators': 300} parameters. F1 score = 0.6009150616190687 and Accuracy score = 0.6009150616190687


Best classifier seems to be Random Forest. Before trying XGBoost let's increase number of estimators and include additional parameters to get even better score from it

In [17]:
# specyfying new parameters for random forest to test
pipeline = [make_pipeline(StandardScaler(),RandomForestClassifier())]
param_grid = [{"randomforestclassifier__criterion":["gini"],
              "randomforestclassifier__n_estimators":[300,500], "randomforestclassifier__min_samples_split":[2,4,6,8,10],
              "randomforestclassifier__min_samples_leaf":[1,2,4,6]}]

In [18]:
# finding best Random Forest model using GridSearchCV
find_best_model(X_train,X_test,y_train,y_test,pipeline,param_grid)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  4.7min
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 19.3min
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed: 29.7min finished


Best model: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())]) with {'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__n_estimators': 500} parameters. F1 score = 0.6012102427865103 and Accuracy score = 0.6012102427865103


In [22]:
# Trying another classifier - XGBoost, using RandomizedSearchCV as XGB has large number of parameters to set and takes fairly long time to compute

pipeline = make_pipeline(StandardScaler(),XGBClassifier())
params = {"xgbclassifier__n_estimators":[10,100,200],
         "xgbclassifier__learning_rate":ss.uniform(0.01,0.3),
          "xgbclassifier__max_depth": ss.randint(5,30),
         "xgbclassifier__min_child_weight":ss.randint(5,50),
         "xgbclassifier__reg_lambda":ss.uniform(0.1,3),
         "xgbclassifier__tree_method":["auto","approx","hist"]}
grid = RandomizedSearchCV(pipeline,
                        param_distributions=params,
                        n_iter = 20,
                        refit=True,      # refitting best model
                        cv = 3,          # nr of crossvalidations
                        verbose=3,       # provides information during the learning process
                        n_jobs = 3)

grid.fit(X_train,y_train)
print(grid.score(X_test,y_test))
y_pred = grid.predict(X_test)
scr_f1 = f1_score(y_test,y_pred,average="micro")
print(scr_f1)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  6.7min
[Parallel(n_jobs=3)]: Done  60 out of  60 | elapsed: 16.9min finished


0.5990701793225592


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [26]:
scr_f1 = f1_score(y_test,y_pred,average="micro")
scr_f1

0.5990701793225592

In [30]:
# retraining best model on whole set, creating confusion matrix to see which class performs worst

model = make_pipeline(StandardScaler(),RandomForestClassifier(criterion="gini",min_samples_split=2,min_samples_leaf=2,n_estimators=500))

model.fit(X,y)
pred = model.predict(X)
conf_matrix = multilabel_confusion_matrix(y,pred)

In [57]:
# confusion matrix is automatically sorted in order of classes. Matching class codes to descriptions, creating dictionary representing matrices for each cause
conf = {}
for i in range(len(conf_matrix)):
    conf[causes[i+1]] = conf_matrix[i]

In [58]:
# confusion matrix is orderd like: [0,0] = True negatives, [0,1] = False negatives, [1.0] = False positives, [1,1] = true positive

conf

{'Lightning': array([[58654,   561],
        [  450,  8088]], dtype=int64),
 'Equipment Use': array([[63123,   111],
        [  919,  3600]], dtype=int64),
 'Smoking': array([[66468,     7],
        [  708,   570]], dtype=int64),
 'Campfire': array([[63714,   238],
        [  657,  3144]], dtype=int64),
 'Debris Burning': array([[44751,  3142],
        [  336, 19524]], dtype=int64),
 'Railroad': array([[67224,     0],
        [  183,   346]], dtype=int64),
 'Arson': array([[60649,   259],
        [  973,  5872]], dtype=int64),
 'Children': array([[66296,    37],
        [  553,   867]], dtype=int64),
 'Miscellaneous': array([[50977,  1803],
        [  864, 14109]], dtype=int64),
 'Fireworks': array([[67156,    25],
        [  109,   463]], dtype=int64),
 'Powerline': array([[66312,     8],
        [  471,   962]], dtype=int64),
 'Structure': array([[67469,     0],
        [  123,   161]], dtype=int64),
 'Missing/Undefined': array([[63807,   245],
        [   90,  3611]], dtype=int64)}

In [62]:
# calculating precission and recall for each class
prec_rec = {}
for key in conf.keys():
    prec_rec[key] = {"Precission":conf[key][1,1]/(conf[key][1,0]+conf[key][1,1]), "Recall":conf[key][1,1]/(conf[key][0,1]+conf[key][1,1])}

In [64]:
prec_rec

{'Lightning': {'Precission': 0.9472944483485594, 'Recall': 0.9351370100589663},
 'Equipment Use': {'Precission': 0.7966364239876079,
  'Recall': 0.9700889248181084},
 'Smoking': {'Precission': 0.4460093896713615, 'Recall': 0.9878682842287695},
 'Campfire': {'Precission': 0.8271507498026835, 'Recall': 0.9296274393849793},
 'Debris Burning': {'Precission': 0.9830815709969789,
  'Recall': 0.8613782758316421},
 'Railroad': {'Precission': 0.6540642722117203, 'Recall': 1.0},
 'Arson': {'Precission': 0.8578524470416362, 'Recall': 0.9577556679171424},
 'Children': {'Precission': 0.6105633802816901, 'Recall': 0.959070796460177},
 'Miscellaneous': {'Precission': 0.942296133039471,
  'Recall': 0.8866892911010558},
 'Fireworks': {'Precission': 0.8094405594405595, 'Recall': 0.9487704918032787},
 'Powerline': {'Precission': 0.6713189113747383, 'Recall': 0.9917525773195877},
 'Structure': {'Precission': 0.5669014084507042, 'Recall': 1.0},
 'Missing/Undefined': {'Precission': 0.97568224804107,
  'Reca

Generally, classification seems to return high recall, as only for two classes the result is lower than 90%. The class is miscellaneous, which does not directly imply the cause and Debris Burning. Even there the recall is over 85%. Generally the model can be employed to find suspicious fire incidents that require investigating by the police. Only in one case, smoking, model is suggesting suspiciousness over twice as often than the true amount of fires caused by smoking. Hovever, smoking is one of the rarer causes of fires, so the number of futile investigations will not be as hard. For most often classes (Lightning, Debris and Miscellaneous) precission is very high so there will not be many futile investigations

Even though the models' F1 and accuracy scores are not very high (around 0.6), under closer examination it seems that recall score, more important in this case, is satisfactory. The model can be employed to mark fires requiring closer investigation